# WNTR Additional Examples
In this notebook, we explore **water quality simulations** in WNTR using concepts similar to those covered in Lecture 19.  
We’ll use the `bwsn2.inp` water network model to demonstrate key examples.

In this exercise, you will:
1. **Define and run** a water age simulation  
2. **Plot time series** of water age at selected junctions  
3. **Analyze the distribution** of water age across the system using a histogram  
4. **Examine relationships** between water age and node degree (e.g., dead-ends) using a box plot
5. **Visualize spatial patterns** of water age across the network
6. **Explore the impact of changing pipe size** on water age

Complete the code where you see 💡


## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

That means that water age and reaction rates are reported in **s**.

# Water Age Simulation 💧

## Import network model

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp = 💡
wn = wntr.network.WaterNetworkModel(inp)

In [ ]:
# check current model settings
# duration
wn.options.time.duration/3600

In [ ]:
# WQ time step
wn.options.time.quality_timestep

In [ ]:
# WQ parameter
💡

## Define and run water age simulation ⚙️

In [ ]:
wn.options.quality.parameter = 💡

In [ ]:
# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()           

In [ ]:
# get results
age = results.node['💡']/3600
age.head()

## Plot water age at selected nodes

In [ ]:
# Plot water age at a few nodes (replace with nodes you discussed in class)
nodes_to_plot = ['JUNCTION-1896', 'JUNCTION-9614', 💡,💡]  # example node IDs 
time_hours = age.index / 3600  # Convert seconds to hours

In [ ]:
plt.plot(time_hours, age[nodes_to_plot], linewidth = 2, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Age (hr)')
plt.title('Water Age')
plt.legend(nodes_to_plot, loc='best') # labels are assigned based on the order that you plot them
plt.show()

## Explore water age across the network

In [ ]:
# let's plot distribution of water ages at all the junctions (excluding tanks, and reservoirs)
junctions = wn.junction_name_list
# junctions[-1]

In [ ]:
# get final age @ all nodes
final_age = age.iloc[-1] 
💡

In [ ]:
# get final age @ all junctions only
final_age_junc = final_age[junctions]
💡

### Distribution of water age across the junctions

In [ ]:
# Plot histogram
plt.hist(💡, bins=20, edgecolor='black')
plt.xlabel('Water age (hours)')
plt.ylabel('Count of junctions')
plt.title('Distribution of water age at final time step')
plt.show()

### Distribution of water age as a function of node degree

node degree = number of links incident to the junction

dead-ends $ \rightarrow $ node degree == 1
`wntr` has a function to get node degree for all nodes. This relies on **NetworkX** Python package for studying complex networks https://networkx.org/

We will plot a box plot that shows the distribution of water ages for different node degrees.  To do this we will:
1. Get node degree for all junctions
2. Get all water ages for all junctions
3. Store this info in a new dataframe
4. Make box-plot

In [ ]:
# Compute node degree from networkx graph ---
G = wn.get_graph()
degree_list = dict(G.degree())
degree_list # this is a list!

In [ ]:
# convert the list to dataframe and get only the degree
degree_df = pd.Series(💡)
degree_junc = degree_df[💡]   # only junctions (without tanks and reservoirs)
degree_junc  # this is a dataframe

In [ ]:
# we can access values only by using
degree_junc.💡

In [ ]:
# water age is already a data frame
final_age_junc.values

In [ ]:
# check that they are the same length
print(len(final_age_junc))
print(len(junctions))

In [ ]:
# Create a DataFrame that combines everything ---
df = pd.DataFrame()
df['Junction'] = 💡
df['Degree'] = 💡
df['WaterAge'] = 💡

In [ ]:
# Box plot of water age by node degree ---
df.boxplot(column = '💡', by = '💡', grid = False)
plt.xlabel('Node degree (# of connected links)')
plt.ylabel('Water age (hours)')
plt.title('Water age distribution by node degree')
plt.suptitle('')  # remove the default pandas subtitle

### Spatial plot of water age across the network 🌍

In [ ]:
wntr.graphics.plot_network(
    wn,
    node_attribute=final_age_junc,
    title='Final Water Age at Junctions (hours)',
    node_cmap='coolwarm',
    node_size=12,
    link_width=0.5,
    add_colorbar=True
)
plt.show()

### Explore the effect of pipe diameter on water age

In [ ]:
# base simulation
#-----------------------------
# set up model
inp = 💡
wn = wntr.network.WaterNetworkModel(inp)

# set AGE
wn.options.quality.parameter = 'AGE'  

# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()  

#get junction list
junctions = wn.junction_name_list

# get final age @ all nodes
age = results.node['quality']/3600
final_age = age.iloc[-1] 

# get final age @ junctions
💡 = final_age[junctions]

In [ ]:
# new simulation
#---------------------------------------
wn = wntr.network.WaterNetworkModel(💡)

# set AGE
wn.options.quality.parameter = 'AGE'

# increase all pipes by 4inch = 0.1016 m
for 💡 in wn.pipe_name_list:
    pipe = wn.get_link(💡)
    pipe.💡 += 0.1016  # +4 inch

# run simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()    # increase all pipe diameters by 4inch ~= 100 mm 

# get final age @ all nodes
age = results.node['quality']/3600
final_age = age.iloc[-1] 

# get final age @ junctions
💡 = final_age[junctions]

In [ ]:
# Histogram overlay to see the shift
plt.hist(💡, bins = 30, alpha = 0.5, label = 'Baseline')
plt.hist(💡, bins = 30, alpha = 0.5, label = '+4in')
plt.xlabel('Final water age (h)'); plt.ylabel('Count'); plt.legend(); plt.show()

In [ ]:
print("Mean water age before/after (h):", age0.mean(), age1.mean())
print("75th percentile before/after (h):",
      np.percentile(age0.dropna(), 75), np.percentile(age1.dropna(), 75))